# Скрипт для классификации текстов из JSON-файлов

Этот скрипт классифицирует тексты новостей из JSON-файлов с использованием модели трансформера. Каждый текст сортируется по заранее определённым категориям (business, finances, political, personal, gaming, science, weather и другие), а результаты сохраняются в исходные файлы.

In [ ]:
# Установка необходимых библиотек
!pip install transformers tqdm torch --quiet

## Импорт библиотек

In [2]:
import re
import os
import json
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

## Настройки

In [ ]:
# Шаблон для классификации текста
PROMPT = """
Question: {question}

Answer:
Ты — система классификации текстов. Классифицируй каждый входящий текст, выбирая одну из следующих категорий. Возвращай **только название категории**, ничего больше.

Категории:

- other
   - Тексты, не подходящие ни под одну из остальных категорий.

- business
   - Новости и информация о бизнесе.
   - Примеры:
      - "Apple выпустила новую модель MacBook с улучшенным процессором".
      - "Компания Tesla построила новый завод в Техасе".

- finances
    - Тексты о финансах, рынках, экономике, деньгах.
    - Примеры:
      - "На фондовом рынке началась коррекция после заявления ФРС".
      - "Центробанк повысил ключевую ставку до 8%".

- political
   - Новости о политике, законах, выборах, государственных органах.
   - Примеры:
     - "Президент подписал указ о введении новых налоговых льгот".
     - "На выборах в США победил кандидат от демократической партии".

- personal
   - Личные происшествия, необычные или курьезные случаи.
   - Примеры:
     - "Мужчина спас собаку, упавшую в замерзшую реку".
     - "В Петербурге кошка случайно заблокировала входную дверь".

- stuff
   - Развлекательные темы, праздники, гороскопы, мистические истории.
   - Примеры:
     - "Сегодня отмечается Всемирный день кофе".
     - "Гороскоп на неделю: Львам нужно быть осторожнее с деньгами".

- gaming
   - Новости про видеоигры, киберспорт, игровые события.
   - Примеры:
     - "Вышло новое обновление для Call of Duty".
     - "Команда Team Spirit победила в финале турнира по Dota 2".

- science
   - Наука, технологии, медицина, исследования.
   - Примеры:
     - "Ученые обнаружили редкий минерал на поверхности Марса".
     - "Новый метод лечения рака показал высокую эффективность".
     - "Ученые разработали новый способ лечения диабета".

- moscow
   - Новости о Москве.
   - Примеры:
     - "В Москве открылась новая линия метро".
     - "На Тверской улице перекроют движение из-за парада".

- weather
   - Новости о погоде: прогнозы, катаклизмы.
   - Примеры:
     - "На этой неделе ожидаются сильные заморозки".
     - "В Краснодарском крае прошел ураган, повредивший несколько зданий".

- it
   - Новости IT и технологий: софт, железо, устройства, языки программирования, машинное обучение.
   - Примеры:
     - "Python остается самым популярным языком программирования в 2024 году".
     - "JavaScript-фреймворк Next.js получил значительные обновления".

- advertisement
    - Рекламные тексты о товарах, услугах, акциях.
    - Примеры:
      - "Скидка 50% на все зимние коллекции в нашем магазине".
      - "Купите новый смартфон со скидкой до конца недели".

---

Классифицируй строго по указанным категориям.
"""

# Список категорий
CATEGORIES = [
    "other",
    "finances",
    "business",
    "political",
    "personal",
    "stuff",
    "gaming",
    "science",
    "moscow",
    "weather",
    "it",
    "advertisement"
]

# Название модели, используемой для загрузки
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

## Работа с моделью

In [ ]:
# Загрузка модели с автоматическим выбором устройства и типа данных
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",  # Автоматический выбор типа тензоров
    device_map="auto"    # Автоматическое распределение по устройствам
)

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

## Вспомогательная функция для очистки текста

In [5]:
def clean_text_from_non_latin_chars(input_text: str) -> str:
    """
    Очищает текст от всех символов, кроме латинских букв (A-Z, a-z) и пробелов.

    Args:
        input_text: Исходный текст для очистки.

    Returns:
        Очищенный текст, содержащий только латинские буквы и пробелы.
    """
    return re.sub(r'[^a-zA-Z\s]', '', input_text)

## Классификация текста с использованием модели

In [ ]:
def classify_text_with_prompt(text: str, model, tokenizer, prompt: str) -> str:
    """
    Классифицирует текст с использованием модели генерации и заданного шаблона.

    Args:
        text: Текст для классификации.
        model: Языковая модель.
        tokenizer: Соответствующий токенизатор.
        prompt: Системный шаблон для классификации.

    Returns:
        Результат классификации.
    """
    # Формируем сообщения для классификации
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": text}
    ]

    # Применяем шаблон и токенизируем
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

    # Генерируем ответ
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=10
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response.strip()

## Обработка и классификация данных

In [ ]:
def process_json_files(json_files: list) -> None:
    """
    Читает данные из JSON-файлов, классифицирует их по категориям и сохраняет результат.

    Args:
        json_files: Список путей к JSON-файлам для обработки.
    """
    for json_file in tqdm(json_files, desc="Обработка файлов"):
        with open(json_file, "r", encoding="utf-8") as file:
            data = json.load(file)

        data = [item for item in data if item["text"]]
        data = data[:1000]  # Берем максимум 1000 записей

        # Классификация каждого элемента данных
        for item in tqdm(data, desc=f"Классификация в {json_file}"):
            if item.get("category") in CATEGORIES:
                continue

            # Классификация текста
            choice = clean_text_from_non_latin_chars(classify_text_with_prompt(item["text"], model, tokenizer, PROMPT))
            item["category"] = choice if choice in CATEGORIES else "error"

        # Сохранение обновленных данных
        with open(json_file, "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

## Считывание и обработка JSON-файлов

In [ ]:
# Получаем список всех JSON-файлов в текущей директории
json_files = [file for file in os.listdir() if file.endswith(".json")]

# Запуск процесса обработки файлов
process_json_files(json_files)